In [25]:
import os
import sys
import re
import numpy as np

### DB 불러오기

In [73]:
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

# db connect 
conn = psycopg2.connect( 
    host = "localhost", 
    dbname = "naver_blog", 
    user = "postgres", 
    password = "1234"
    )         
sslmode = "require"
print("Connection established")

cursor = conn.cursor()

# DB에 저장된 테이블과 모든 열과 행을 가져오기
cursor.execute("SELECT * FROM 캠핑;")
result = cursor.fetchall()
# fetchall() 메서드는 모든 데이터를 한꺼번에 클라이언트로 가져올 때 사용
# fetchone()은 한번 호출에 하나의 Row 만을 가져올 때 사용됨. fetchone()을 여러 번 호출하면, 호출 때마다 한 Row씩 데이터를 가져옴 
# fetchmany(n) 메서드는 n개 만큼의 데이터를 한꺼번에 가져올 때 사용

# 데이터프레임으로 만들기
캠핑2018_19_df = pd.DataFrame(result)
# row를 불러오는 방법1
캠핑2018_19_df.columns = [i[0] for i in  cursor.description]
캠핑2018_19_df 

# row를 불러오는 방법2
# Print all rows
# for row in rows:
#     Data_row = "(%s, %s, %s, %s)" % (str(row[0]), str(row[1]), str(row[2]), str(row[3]))

# Cleanup
# conn.commit()
# cursor.close()
# conn.close()

Connection established


,id,제목,링크,발행일자,본문
0,0,폴딩 텐트 트레일러 겨울캠핑도 문제없어 !,https://blog.naver.com/alswh03/221197806276,2018-01-31,(18)\n공지 카카오뷰 창작센터 개설하고 보드 발행 방법 [카뷰 채널친구 맞추 단...
1,1,캠핑카와 캠핑트레일러 제작 믿을수있는 국내 업체,https://blog.naver.com/gasinae00/221189063401,2018-01-21,공지 EBS 다큐 잇 it 캠핑방송 출연 2박3일간 촬영 힘들었지만 방송보니 좋네\...
2,2,감바스 알 아히요 캠핑 파티요리 만들기~,https://blog.naver.com/laeda/221176408914,2018-01-02,(81)\n공지 사진 무단 도용시 법적 대응하겠습니다!\n2021. 2. 22.\n...
3,3,남양주 별내 짱아캠핑장에서 장박캠핑 시작,https://blog.naver.com/sks0926/221190316394,2018-01-22,"캠핑,여행,낚시이야기 253개의 글\n목록닫기\n글 제목\n작성일\n홍천강 노지캠핑..."
4,4,캠핑카제작 업체 믿고 맡기는 캠카랜드 놀라움!,https://blog.naver.com/dull33/221197657196,2018-01-31,(7)\n공지 네이버TV 채널 만들기 구독과 좋아요 부탁드려요^^\n2018. 7....
...,...,...,...,...,...
25195,25195,"134th Camping, in 귤빛캠핑장... 장박캠핑 3주 차",https://blog.naver.com/heoung00/221754335956,2019-12-30,2019년 12월 27일(금) ~ 29일(일)\n장박지인 귤빛캠핑장에서 장박 3주 ...
25196,25196,2019년 구니의 캠핑 & 여행 결산,https://blog.naver.com/jtong/221754488506,2019-12-30,캠핑과 여행을 좋아하는 구니입니다.\n2019년도 어느덧 마무리가 되어가는 12월입...
25197,25197,스타렉스 세미캠핑카 스카이라이프SLT 시공설치~ 스카이오토,https://blog.naver.com/ancobang/221747648096,2019-12-24,오늘은 스타렉스 차량을 캠핑카로 개조하신 차량이 입고가되어\n스카이라이프SLT(LT...
25198,25198,[캠핑] 차박 캠핑 / 강릉 캠핑,https://blog.naver.com/noravose/221755331578,2019-12-31,2019.12.14 - 15\n선자령 백패킹을 마치고 샤워 후 2일차 캠핑을 위해 ...


### 전처리

In [30]:
# # id값을 인덱스로 바꾸기
# def id_index(df):
#     df2 = df.copy()
#     df2 = df2.set_index("id")
#     return df2

In [ ]:
# np.nan으로 결측값을 인식할 때 해결
# df['본문'].replace('', np.nan, inplace=True)
# df

In [74]:
def nomalize(df):
    df2 = df.copy()
    # 전체 결측 데이터 확인
    print('드롭 전 행의 총 개수 :', len(df2))
    print('결측값 총합 :', df2.isnull().sum().sum())
    # 결측값이 있는 행 없애기
    df2 = df2.dropna(axis=0)
    print('드롭 후 행의 총 개수 :', len(df2))
    print('결측값 총합 :', df2.isnull().sum().sum())
    print('중복값 확인 :', df2.duplicated().sum().sum())
    df2 = df2.drop_duplicates(ignore_index=True)
    print('총 행의 개수 :', len(df2))
    return df2

In [83]:
# 한글과 공백 제외 모두 제거 : 띄어쓰기 유지, 구두점 제거
def extract_word(context):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', context)
    return result

In [97]:
def drop_row(df2):
    df3 = df2.copy()
    df3 = df3.drop(df3.columns[[0, 2, 3, 4]], axis=1)
    return df3

,제목
0,폴딩 텐트 트레일러 겨울캠핑도 문제없어
1,캠핑카와 캠핑트레일러 제작 믿을수있는 국내 업체
2,감바스 알 아히요 캠핑 파티요리 만들기
3,남양주 별내 짱아캠핑장에서 장박캠핑 시작
4,캠핑카제작 업체 믿고 맡기는 캠카랜드 놀라움


### 데이터프레임 csv파일로 저장

In [98]:
# 데이터프레임 csv파일로 저장
def save_csv(df, name):
    name = name
    df.to_csv(f'{name}.csv', index=False, encoding='cp949')

### main 함수

In [ ]:
# 메인 함수 : 함수 한꺼번에 실행
if __name__=='__main__':
    df = 캠핑2018_19_df
    df2 = nomalize(df)
    df2['제목'] = df2['제목'].apply(lambda x:extract_word(x))
    df2['본문'] = df2['본문'].apply(lambda x:extract_word(x))
    df3 = drop_row(df2)
    df = df3
    name = input('파일 이름을 지어주세요: ')
    save_csv(df, name)
    
df2.head()
df3.head()